In [ ]:
# !pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 79.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 106.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 16.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.14.1
    Uninstalling scipy-1.14.1:
      Successfully uninstalled scipy-1.14.1


In [1]:
!pip uninstall -y gensim
!pip install gensim --no-cache-dir

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 83.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 279.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 263.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 280.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.14.1
    Uninstalling scipy-1.14.1:
      Successfully uninstalled scipy-1.14.1


In [1]:
import pandas as pd
import numpy as np
import string

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk import pos_tag

from gensim.models import KeyedVectors

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score
from keras.preprocessing.sequence import pad_sequences

import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim

In [4]:
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


True

#### **Step 1: Loading the dataset and splitting as train and test.** ####

- I wanted to proceed with one of the benchmark datasets, IMDB.

In [5]:
## https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

imdb_df = pd.read_csv("/content/sample_data/IMDB Dataset.csv")

# Labeling sentiments to 0 and 1 for better handling and dropping the sentiment column.
imdb_df['label'] = imdb_df['sentiment'].map({'positive': 1, 'negative': 0})
imdb_df.drop(columns='sentiment', inplace=True)

print(imdb_df.head())

                                              review  label
0  One of the other reviewers has mentioned that ...      1
1  A wonderful little production. <br /><br />The...      1
2  I thought this was a wonderful way to spend ti...      1
3  Basically there's a family where a little boy ...      0
4  Petter Mattei's "Love in the Time of Money" is...      1


In [6]:
imdb_df.shape

(50000, 2)

In [7]:
print(imdb_df['label'].value_counts())

label
1    25000
0    25000
Name: count, dtype: int64


In [8]:
## Confirming that the dataset does not have a null value.
print(imdb_df.isnull().sum())

review    0
label     0
dtype: int64


In [9]:
## Splitting the dataset as train (%80) and test (%20)

imdb_df_train, imdb_df_test = train_test_split(imdb_df, test_size=0.2, random_state=42)

print("Train size --> {}\nTest size  --> {}".format(len(imdb_df_train), len(imdb_df_test)))

Train size --> 40000
Test size  --> 10000


#### **Step 2: Text Preprocessing** ####

In [10]:
## First of all, lowercasing the reviews.

imdb_df_train_lowercase = []
for review in imdb_df_train['review']:
    imdb_df_train_lowercase.append(review.lower())

imdb_df_test_lowercase = []
for review in imdb_df_test['review']:
    imdb_df_test_lowercase.append(review.lower())

In [11]:
imdb_df_train_lowercase[0]

'that\'s what i kept asking myself during the many fights, screaming matches, swearing and general mayhem that permeate the 84 minutes. the comparisons also stand up when you think of the one-dimensional characters, who have so little depth that it is virtually impossible to care what happens to them. they are just badly written cyphers for the director to hang his multicultural beliefs on, a topic that has been done much better in other dramas both on tv and the cinema.<br /><br />i must confess, i\'m not really one for spotting bad performances during a film, but it must be said that nichola burley (as the heroine\'s slutty best friend) and wasim zakir (as the nasty, bullying brother) were absolutely terrible. i don\'t know what acting school they graduated from, but if i was them i\'d apply for a full refund post haste. only samina awan in the lead role manages to impress in a cast of so-called british talent that we\'ll probably never hear from again. at least, that\'s the hope. ne

In [12]:
## https://www.geeksforgeeks.org/python-remove-punctuation-from-string/

## Removing punctuation

imdb_df_train_no_punct = []
for review in imdb_df_train_lowercase:
    clean = ""
    for char in review:
        if char not in string.punctuation:
            clean += char
    imdb_df_train_no_punct.append(clean)

imdb_df_test_no_punct = []
for review in imdb_df_test_lowercase:
    clean = ""
    for char in review:
        if char not in string.punctuation:
            clean += char
    imdb_df_test_no_punct.append(clean)

In [13]:
imdb_df_train_no_punct[0]

'thats what i kept asking myself during the many fights screaming matches swearing and general mayhem that permeate the 84 minutes the comparisons also stand up when you think of the onedimensional characters who have so little depth that it is virtually impossible to care what happens to them they are just badly written cyphers for the director to hang his multicultural beliefs on a topic that has been done much better in other dramas both on tv and the cinemabr br i must confess im not really one for spotting bad performances during a film but it must be said that nichola burley as the heroines slutty best friend and wasim zakir as the nasty bullying brother were absolutely terrible i dont know what acting school they graduated from but if i was them id apply for a full refund post haste only samina awan in the lead role manages to impress in a cast of socalled british talent that well probably never hear from again at least thats the hope next time hire a different scoutbr br anothe

In [14]:
## Tokenizing the text into words/tokens

imdb_train_tokenized = []
for review in imdb_df_train_no_punct:
    tokens = word_tokenize(review)
    imdb_train_tokenized.append(tokens)

imdb_test_tokenized = []
for review in imdb_df_test_no_punct:
    tokens = word_tokenize(review)
    imdb_test_tokenized.append(tokens)

In [15]:
imdb_train_tokenized[0]

['thats',
 'what',
 'i',
 'kept',
 'asking',
 'myself',
 'during',
 'the',
 'many',
 'fights',
 'screaming',
 'matches',
 'swearing',
 'and',
 'general',
 'mayhem',
 'that',
 'permeate',
 'the',
 '84',
 'minutes',
 'the',
 'comparisons',
 'also',
 'stand',
 'up',
 'when',
 'you',
 'think',
 'of',
 'the',
 'onedimensional',
 'characters',
 'who',
 'have',
 'so',
 'little',
 'depth',
 'that',
 'it',
 'is',
 'virtually',
 'impossible',
 'to',
 'care',
 'what',
 'happens',
 'to',
 'them',
 'they',
 'are',
 'just',
 'badly',
 'written',
 'cyphers',
 'for',
 'the',
 'director',
 'to',
 'hang',
 'his',
 'multicultural',
 'beliefs',
 'on',
 'a',
 'topic',
 'that',
 'has',
 'been',
 'done',
 'much',
 'better',
 'in',
 'other',
 'dramas',
 'both',
 'on',
 'tv',
 'and',
 'the',
 'cinemabr',
 'br',
 'i',
 'must',
 'confess',
 'im',
 'not',
 'really',
 'one',
 'for',
 'spotting',
 'bad',
 'performances',
 'during',
 'a',
 'film',
 'but',
 'it',
 'must',
 'be',
 'said',
 'that',
 'nichola',
 'burley

In [16]:
len(imdb_train_tokenized[0])

400

In [17]:
## https://www.geeksforgeeks.org/removing-stop-words-nltk-python/

## Removing stop words

stop_words = set(stopwords.words('english'))

imdb_train_filtered = []
for tokens in imdb_train_tokenized:
    filtered_words = []
    for word in tokens:
        if word not in stop_words:
            filtered_words.append(word)
    imdb_train_filtered.append(filtered_words)

imdb_test_filtered = []
for tokens in imdb_test_tokenized:
    filtered_words = []
    for word in tokens:
        if word not in stop_words:
            filtered_words.append(word)
    imdb_test_filtered.append(filtered_words)

In [18]:
imdb_train_filtered[0]

['thats',
 'kept',
 'asking',
 'many',
 'fights',
 'screaming',
 'matches',
 'swearing',
 'general',
 'mayhem',
 'permeate',
 '84',
 'minutes',
 'comparisons',
 'also',
 'stand',
 'think',
 'onedimensional',
 'characters',
 'little',
 'depth',
 'virtually',
 'impossible',
 'care',
 'happens',
 'badly',
 'written',
 'cyphers',
 'director',
 'hang',
 'multicultural',
 'beliefs',
 'topic',
 'done',
 'much',
 'better',
 'dramas',
 'tv',
 'cinemabr',
 'br',
 'must',
 'confess',
 'im',
 'really',
 'one',
 'spotting',
 'bad',
 'performances',
 'film',
 'must',
 'said',
 'nichola',
 'burley',
 'heroines',
 'slutty',
 'best',
 'friend',
 'wasim',
 'zakir',
 'nasty',
 'bullying',
 'brother',
 'absolutely',
 'terrible',
 'dont',
 'know',
 'acting',
 'school',
 'graduated',
 'id',
 'apply',
 'full',
 'refund',
 'post',
 'haste',
 'samina',
 'awan',
 'lead',
 'role',
 'manages',
 'impress',
 'cast',
 'socalled',
 'british',
 'talent',
 'well',
 'probably',
 'never',
 'hear',
 'least',
 'thats',
 'h

In [19]:
len(imdb_train_filtered[0])

214

- The following code snippet is about cleaning the tokens from the non-alphabetical tokens because when I was handling word_to_idx, I noticed that I was getting weird tokens for the vocabulary, so I wanted to solve that by removing these tokens.

In [20]:
imdb_train_alphabetical = []
for tokens in imdb_train_filtered:
    alphabetical_words = []
    for word in tokens:
        if word.isalpha():
            alphabetical_words.append(word)
    imdb_train_alphabetical.append(alphabetical_words)

imdb_test_alphabetical = []
for tokens in imdb_test_filtered:
    alphabetical_words = []
    for word in tokens:
        if word.isalpha():
            alphabetical_words.append(word)
    imdb_test_alphabetical.append(alphabetical_words)

- While handling the lemmatization, I was not receiving proper lemmas because I was not doing POS tagging. Then, I researched online and found how to solve it with NLTK library.

In [21]:
## https://stackoverflow.com/questions/15586721/wordnet-lemmatization-and-pos-tagging-in-python

## I was receiving Key Error when I followed the link above for the last line, return ''
## Since WordNetLemmatizer only supports 4 POS tags, if there is no match, I returned Noun.

def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [22]:
## https://www.nltk.org/api/nltk.stem.WordNetLemmatizer.html?highlight=wordnet
## https://www.geeksforgeeks.org/python-lemmatization-with-nltk/

## Lemmatizing the tokens

lemmatizer = WordNetLemmatizer()

imdb_train_lemmatized = []
for tokens in imdb_train_alphabetical:
    tagged_tokens = pos_tag(tokens)
    lemma_list = []
    for word, tag in tagged_tokens:
        wordnet_tag = get_wordnet_pos(tag)
        lemma = lemmatizer.lemmatize(word, pos=wordnet_tag)
        lemma_list.append(lemma)
    imdb_train_lemmatized.append(lemma_list)

imdb_test_lemmatized = []
for tokens in imdb_test_alphabetical:
    tagged_tokens = pos_tag(tokens)
    lemma_list = []
    for word, tag in tagged_tokens:
        wordnet_tag = get_wordnet_pos(tag)
        lemma = lemmatizer.lemmatize(word, pos=wordnet_tag)
        lemma_list.append(lemma)
    imdb_test_lemmatized.append(lemma_list)

In [23]:
imdb_train_lemmatized[0]

['thats',
 'keep',
 'ask',
 'many',
 'fight',
 'scream',
 'match',
 'swear',
 'general',
 'mayhem',
 'permeate',
 'minute',
 'comparison',
 'also',
 'stand',
 'think',
 'onedimensional',
 'character',
 'little',
 'depth',
 'virtually',
 'impossible',
 'care',
 'happen',
 'badly',
 'write',
 'cypher',
 'director',
 'hang',
 'multicultural',
 'belief',
 'topic',
 'do',
 'much',
 'well',
 'dramas',
 'tv',
 'cinemabr',
 'br',
 'must',
 'confess',
 'im',
 'really',
 'one',
 'spot',
 'bad',
 'performance',
 'film',
 'must',
 'say',
 'nichola',
 'burley',
 'heroine',
 'slutty',
 'best',
 'friend',
 'wasim',
 'zakir',
 'nasty',
 'bullying',
 'brother',
 'absolutely',
 'terrible',
 'dont',
 'know',
 'act',
 'school',
 'graduate',
 'id',
 'apply',
 'full',
 'refund',
 'post',
 'haste',
 'samina',
 'awan',
 'lead',
 'role',
 'manages',
 'impress',
 'cast',
 'socalled',
 'british',
 'talent',
 'well',
 'probably',
 'never',
 'hear',
 'least',
 'thats',
 'hope',
 'next',
 'time',
 'hire',
 'differe

- The above result is not perfect for sure, but I still wanted to lemmatize.

- I wanted to go with word2vec because it is the used method for LSTM example in the class. I will try to follow that.

In [24]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [25]:
## https://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.KeyedVectors.load_word2vec_format
## https://radimrehurek.com/gensim/auto_examples/tutorials/run_word2vec.html
## https://radimrehurek.com/gensim/models/word2vec.html
## https://www.geeksforgeeks.org/python-word-embedding-using-word2vec/
## https://spotintelligence.com/2023/02/15/word2vec-for-text-classification/
## https://github.com/clairett/pytorch-sentiment-classification/tree/master

path = "/content/drive/My Drive/GoogleNews-vectors-negative300.bin"

## Loading the pretrained model as the sample.
word2vec_model = KeyedVectors.load_word2vec_format(path, binary=True)

In [26]:
## Confirming if the model works properly.
len(word2vec_model["king"])

300

In [29]:
## I am trying to follow the sample code's method to understand properly.
## So, here, I am creating my own vocabulary from the lemmatized training data.

imdb_vocab = set()
for tokens in imdb_train_lemmatized:
    for word in tokens:
        imdb_vocab.add(word)

## Sorting the vocabulary and creating the word-to-index dictionary as the sample.
## I left the index 0 for the padding for LSTM.
imdb_vocab = sorted(list(imdb_vocab))
word_to_idx = {}
for i in range(len(imdb_vocab)):
    word_to_idx[imdb_vocab[i]] = i + 1

In [30]:
word_to_idx

{'aa': 1,
 'aaa': 2,
 'aaaaaaaaaaaahhhhhhhhhhhhhh': 3,
 'aaaaaaaargh': 4,
 'aaaaagh': 5,
 'aaaaah': 6,
 'aaaaargh': 7,
 'aaaaatchkah': 8,
 'aaaaaw': 9,
 'aaaahhhhhh': 10,
 'aaaahhhhhhh': 11,
 'aaaand': 12,
 'aaaarrgh': 13,
 'aaaawwwwww': 14,
 'aaah': 15,
 'aaahhhhhhh': 16,
 'aaahthe': 17,
 'aaand': 18,
 'aaargh': 19,
 'aaarrrghim': 20,
 'aaaugh': 21,
 'aab': 22,
 'aachen': 23,
 'aada': 24,
 'aadha': 25,
 'aadmittedly': 26,
 'aag': 27,
 'aage': 28,
 'aaghh': 29,
 'aah': 30,
 'aahed': 31,
 'aahhh': 32,
 'aahhhh': 33,
 'aaila': 34,
 'aailiyah': 35,
 'aaja': 36,
 'aajala': 37,
 'aakash': 38,
 'aake': 39,
 'aaker': 40,
 'aalcc': 41,
 'aaliyah': 42,
 'aaliyahs': 43,
 'aalox': 44,
 'aames': 45,
 'aamesbr': 46,
 'aamess': 47,
 'aamilne': 48,
 'aamir': 49,
 'aamirbr': 50,
 'aamirs': 51,
 'aamirsalmanraveenakarishma': 52,
 'aamr': 53,
 'aan': 54,
 'aankh': 55,
 'aankhen': 56,
 'aap': 57,
 'aapke': 58,
 'aapkey': 59,
 'aaran': 60,
 'aardman': 61,
 'aardmanbr': 62,
 'aardmans': 63,
 'aardvark': 64

In [31]:
imdb_vocab

['aa',
 'aaa',
 'aaaaaaaaaaaahhhhhhhhhhhhhh',
 'aaaaaaaargh',
 'aaaaagh',
 'aaaaah',
 'aaaaargh',
 'aaaaatchkah',
 'aaaaaw',
 'aaaahhhhhh',
 'aaaahhhhhhh',
 'aaaand',
 'aaaarrgh',
 'aaaawwwwww',
 'aaah',
 'aaahhhhhhh',
 'aaahthe',
 'aaand',
 'aaargh',
 'aaarrrghim',
 'aaaugh',
 'aab',
 'aachen',
 'aada',
 'aadha',
 'aadmittedly',
 'aag',
 'aage',
 'aaghh',
 'aah',
 'aahed',
 'aahhh',
 'aahhhh',
 'aaila',
 'aailiyah',
 'aaja',
 'aajala',
 'aakash',
 'aake',
 'aaker',
 'aalcc',
 'aaliyah',
 'aaliyahs',
 'aalox',
 'aames',
 'aamesbr',
 'aamess',
 'aamilne',
 'aamir',
 'aamirbr',
 'aamirs',
 'aamirsalmanraveenakarishma',
 'aamr',
 'aan',
 'aankh',
 'aankhen',
 'aap',
 'aapke',
 'aapkey',
 'aaran',
 'aardman',
 'aardmanbr',
 'aardmans',
 'aardvark',
 'aarf',
 'aargh',
 'aarika',
 'aaron',
 'aaronbr',
 'aaroncurb',
 'aaronjd',
 'aarp',
 'aashok',
 'aasize',
 'aasmaan',
 'aasman',
 'aatish',
 'aaton',
 'aau',
 'aauugghh',
 'aavjo',
 'aawip',
 'aaww',
 'ab',
 'aba',
 'aback',
 'abagail',
 'aba

- Even though there are still weird tokens, I cleaned and removed the weirdest ones

In [32]:
## Building the embbedding matrix by firstly random initialzation for all words,
## then setting the vector at index 0 to zeros for padding, and then overwriting
## the real vectors from Word2Vec.

imdb_vocab_size = len(imdb_vocab) + 1

pretrained_embeddings = np.random.uniform(-0.25, 0.25, (imdb_vocab_size, 300))

pretrained_embeddings[0] = np.zeros(300)

real_vectors = 0
for word, idx in word_to_idx.items():
    if word in word2vec_model:
        pretrained_embeddings[idx] = word2vec_model[word]
        real_vectors += 1

print("There is {} Word2Vec vectors out of {} total IMDB vocabulary words.".format(real_vectors, imdb_vocab_size))

There is 47568 Word2Vec vectors out of 143522 total IMDB vocabulary words.


In [33]:
## https://medium.com/@reddyyashu20/build-text-classification-model-using-word2vec-nlp-part2-52aa2839e8f4
## https://www.kaggle.com/code/namyang94/imdb-sentiment-classification-methods

## After the research, I gathered that I need to average vectors for traditional
## ML model by converting each review into a 300-dim feature vector. So, Word2Vec
## gave me 300-dim vector for each word -which is what the sample code does and
## it works for LSTM-, but I need one one 300-dim vector per review for Logistic
## Regression and I am doing that by taking average.

imdb_train_vectors_avg = []
for tokens in imdb_train_lemmatized:
    vectors = []
    for word in tokens:
        if word in word2vec_model:
            vectors.append(word2vec_model[word])
    if len(vectors) > 0:
        avg_vector = np.mean(vectors, axis=0)
    else:
        avg_vector = np.zeros(300)
    imdb_train_vectors_avg.append(avg_vector)

imdb_test_vectors_avg = []
for tokens in imdb_test_lemmatized:
    vectors = []
    for word in tokens:
        if word in word2vec_model:
            vectors.append(word2vec_model[word])
    if len(vectors) > 0:
        avg_vector = np.mean(vectors, axis=0)
    else:
        avg_vector = np.zeros(300)
    imdb_test_vectors_avg.append(avg_vector)

In [34]:
imdb_train_vectors_avg[0]

array([ 5.45742922e-02,  1.61846559e-02,  1.04651786e-03,  8.44756216e-02,
       -5.92256747e-02,  2.73378547e-02,  6.51020110e-02, -6.08244427e-02,
        8.19856822e-02,  6.78403825e-02, -3.53200883e-02, -1.27185091e-01,
       -2.57275421e-02,  4.43600751e-02, -1.12662576e-01,  1.00253247e-01,
        4.38146479e-02,  1.13928519e-01, -7.33057642e-03, -8.67346525e-02,
       -2.29286426e-03,  4.60926853e-02,  7.65669048e-02,  3.45746428e-03,
        5.04747443e-02, -3.55456881e-02, -7.85460845e-02,  4.10671830e-02,
        2.07941551e-02, -7.33971130e-03, -7.33301565e-02,  2.19822899e-02,
       -4.04082388e-02, -4.14172094e-03,  7.59471513e-05, -2.26715412e-02,
        5.49776964e-02,  4.89208512e-02,  3.54861058e-02,  7.84990415e-02,
        1.04250669e-01, -4.94368859e-02,  9.99694988e-02, -1.54750133e-02,
       -3.04152165e-02, -4.96684387e-02, -5.76231293e-02, -5.24910027e-03,
        1.59848724e-02,  2.01292131e-02, -3.81132178e-02,  3.87399457e-02,
       -2.83316728e-02, -

- Word embedding part with word2vec took a lot of time, but after all those research and analysis about the example code, I was able to do that.

#### **Step 3 and Step 4: Modeling and Performance Evaluation** ####

- I chose to proceed with Logistic Regression for ML and LSTM for DL models.

In [35]:
## Logistic Regression

X_train_log_reg = np.array(imdb_train_vectors_avg)
X_test_log_reg = np.array(imdb_test_vectors_avg)

y_train_log_reg = imdb_df_train['label'].values
y_test_log_reg = imdb_df_test['label'].values

log_reg_model = LogisticRegression()
log_reg_model.fit(X_train_log_reg, y_train_log_reg)

LogisticRegression()

In [36]:
y_pred_log_reg = log_reg_model.predict(X_test_log_reg)

print("\n--- Performance of Logistic Regression ---\n")
print("Precision -->", precision_score(y_test_log_reg, y_pred_log_reg))
print("Recall    -->", recall_score(y_test_log_reg, y_pred_log_reg))
print("F1 Score  -->", f1_score(y_test_log_reg, y_pred_log_reg))


--- Performance of Logistic Regression ---

Precision --> 0.8447082096933729
Recall    --> 0.8473903552292121
F1 Score  --> 0.8460471567267683


- Not using torch_text becomes heavily expensive for me because now I need to manually convert the text into padded index sequences to replicate "text_field.vocab.stoi = word_to_idx" for LSTM network. I was receiving error for torch_text library while uploading the dataset, so I needed to upload and prepare the dataset manually by downloading from Kaggle

In [37]:
## https://www.tensorflow.org/text/guide/tokenizers

## Converting tokens into index sequences to feed LSTM because it takes integer sequences.
## If it does not find an index, then I use 0.

imdb_train_idx_seq = []
for tokens in imdb_train_lemmatized:
    sequence = []
    for word in tokens:
        idx = word_to_idx.get(word, 0)
        sequence.append(idx)
    imdb_train_idx_seq.append(sequence)

imdb_test_idx_seq = []
for tokens in imdb_test_lemmatized:
    sequence = []
    for word in tokens:
        idx = word_to_idx.get(word, 0)
        sequence.append(idx)
    imdb_test_idx_seq.append(sequence)

In [38]:
## https://www.tensorflow.org/api_docs/python/tf/keras/utils/pad_sequences
## https://stackoverflow.com/questions/42943291/what-does-keras-io-preprocessing-sequence-pad-sequences-do

## Here, I am padding the sequnces because I need vectors with uniform length.
## Then, preparing the label arrays to pass into LSTM.

X_train_lstm_padded = pad_sequences(imdb_train_idx_seq, maxlen=300, padding='post')
X_test_lstm_padded = pad_sequences(imdb_test_idx_seq, maxlen=300, padding='post')

y_train_lstm = np.array(imdb_df_train['label'].values, dtype=np.float32)
y_test_lstm = np.array(imdb_df_test['label'].values, dtype=np.float32)

In [39]:
## LSTM Architecture, again, I tried to stick with the sample code but my
## pipeline is different, so not everything stays same.

class LSTMSentiment(nn.Module):
    def __init__(self, embedding_matrix, hidden_dim=256):
        super(LSTMSentiment, self).__init__()
        vocab_size, embedding_dim = embedding_matrix.shape
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.embedding.weight = nn.Parameter(torch.tensor(embedding_matrix, dtype=torch.float32))
        self.embedding.weight.requires_grad = False
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.hidden2label = nn.Linear(hidden_dim, 1)

    def init_hidden(self, batch_size):
        return (torch.zeros(1, batch_size, self.hidden_dim),
                torch.zeros(1, batch_size, self.hidden_dim))

    def forward(self, sentence, hidden):
        embeddings = self.embedding(sentence)
        lstm_out, hidden = self.lstm(embeddings, hidden)
        last_output = lstm_out[-1]
        out = self.hidden2label(last_output)
        return torch.sigmoid(out).squeeze(1), hidden

In [44]:
## https://pytorch.org/tutorials/beginner/basics/data_tutorial.html
## https://discuss.pytorch.org/t/what-do-tensordataset-and-dataloader-do/107017
## https://pytorch.org/docs/stable/data.html

## Here, creating dataloaders for training and testing because I need to handle manually
## To do that, TensorDataset combines inputs and labels; DataLoader handles batching

X_train_lstm_tensor = torch.tensor(X_train_lstm_padded, dtype=torch.long)
X_test_lstm_tensor = torch.tensor(X_test_lstm_padded, dtype=torch.long)

y_train_lstm_tensor = torch.tensor(y_train_lstm, dtype=torch.float32)
y_test_lstm_tensor = torch.tensor(y_test_lstm, dtype=torch.float32)

lstm_train_data = TensorDataset(X_train_lstm_tensor, y_train_lstm_tensor)
lstm_test_data = TensorDataset(X_test_lstm_tensor, y_test_lstm_tensor)

lstm_train_loader = DataLoader(lstm_train_data, batch_size=64, shuffle=True)
lstm_test_loader = DataLoader(lstm_test_data, batch_size=64)

In [51]:
print("CUDA Available -->", torch.cuda.is_available())
print("Current Device -->", torch.cuda.current_device())
print("Device Name    -->", torch.cuda.get_device_name(0))

CUDA Available --> True
Current Device --> 0
Device Name    --> Tesla T4


In [47]:
## https://www.geeksforgeeks.org/long-short-term-memory-networks-using-pytorch/
## https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html

## Training LSTM

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

model = LSTMSentiment(pretrained_embeddings).to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

cuda


In [48]:
epochs = 50
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for inputs, labels in lstm_train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        inputs = inputs.permute(1, 0)

        hidden = model.init_hidden(inputs.shape[1])
        h_0, c_0 = hidden[0].to(device), hidden[1].to(device)
        hidden = (h_0, c_0)

        optimizer.zero_grad()
        outputs, hidden = model(inputs, hidden)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print("Epoch {}/{} - Loss --> {:.4f}".format(epoch+1, epochs, total_loss))

Epoch 1/50 - Loss --> 432.3031
Epoch 2/50 - Loss --> 404.6075
Epoch 3/50 - Loss --> 389.2106
Epoch 4/50 - Loss --> 389.2827
Epoch 5/50 - Loss --> 417.5304
Epoch 6/50 - Loss --> 386.1023
Epoch 7/50 - Loss --> 376.3022
Epoch 8/50 - Loss --> 369.9109
Epoch 9/50 - Loss --> 222.6037
Epoch 10/50 - Loss --> 196.9735
Epoch 11/50 - Loss --> 184.1229
Epoch 12/50 - Loss --> 202.8967
Epoch 13/50 - Loss --> 194.9883
Epoch 14/50 - Loss --> 146.4446
Epoch 15/50 - Loss --> 132.4649
Epoch 16/50 - Loss --> 114.1491
Epoch 17/50 - Loss --> 94.8872
Epoch 18/50 - Loss --> 77.8143
Epoch 19/50 - Loss --> 66.5989
Epoch 20/50 - Loss --> 51.3851
Epoch 21/50 - Loss --> 43.0886
Epoch 22/50 - Loss --> 37.4867
Epoch 23/50 - Loss --> 32.4083
Epoch 24/50 - Loss --> 31.3223
Epoch 25/50 - Loss --> 25.2791
Epoch 26/50 - Loss --> 26.9831
Epoch 27/50 - Loss --> 22.0892
Epoch 28/50 - Loss --> 21.5223
Epoch 29/50 - Loss --> 19.6698
Epoch 30/50 - Loss --> 18.0054
Epoch 31/50 - Loss --> 18.0514
Epoch 32/50 - Loss --> 16.9821
E

In [50]:
## Performance evaluation for LSTM

model.eval()

all_preds = []
all_labels = []
with torch.no_grad():
    for inputs, labels in lstm_test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        inputs = inputs.permute(1, 0)
        hidden = model.init_hidden(inputs.shape[1])
        h_0, c_0 = hidden[0].to(device), hidden[1].to(device)
        hidden = (h_0, c_0)

        outputs, hidden = model(inputs, hidden)
        preds = (outputs > 0.5).int().cpu().numpy()
        all_preds.extend(preds)
        all_labels.extend(labels.cpu().numpy())


precision = precision_score(all_labels, all_preds)
recall = recall_score(all_labels, all_preds)
f1 = f1_score(all_labels, all_preds)

print("\n--- Performance of LSTM Model ---\n")
print("Precision -->", precision)
print("Recall    -->", recall)
print("F1 Score  -->", f1)


--- Performance of LSTM Model ---

Precision --> 0.8648275862068966
Recall    --> 0.8710061520142885
F1 Score  --> 0.8679058730472612


- That's the end of the implementation and results. My findings with the analysis including all necessary parts can be found in my report; I tried not to write too much here.